In [10]:
import pandas as pd
from scipy.stats import spearmanr
import gensim

# Load SimLex dataset
simlex_data = pd.read_csv('/home/ma/ma_ma/ma_sguliyev/New_thesis_repository/Data/SimLex-999/SimLex-999.txt', sep='\t')

# Load the model
model = gensim.models.Word2Vec.load('/home/ma/ma_ma/ma_sguliyev/New_thesis_repository/Notebooks/sampled_word2vec_model_100_percent.model')

# Modified evaluation function
def evaluate_model(model, top_n=10):
    pairs_data = []
    for i, row in simlex_data.iterrows():
        word1, word2, simlex_score = row['word1'], row['word2'], row['SimLex999']
        if word1 in model.wv and word2 in model.wv:
            model_score = model.wv.similarity(word1, word2)
            pairs_data.append((word1, word2, simlex_score, model_score))

    # Convert to DataFrame for easy processing
    df = pd.DataFrame(pairs_data, columns=['word1', 'word2', 'simlex_score', 'model_score'])
    
    # Calculate ranks
    df['simlex_rank'] = df['simlex_score'].rank(ascending=False)
    df['model_rank'] = df['model_score'].rank(ascending=False)

    # Calculate the absolute difference in ranks
    df['rank_difference'] = abs(df['simlex_rank'] - df['model_rank'])

    # Sort by smallest rank differences
    df = df.sort_values(by='rank_difference')

    # Return top N pairs
    return df.head(top_n)

# Evaluate the model and get top N word pairs
top_n_pairs = evaluate_model(model, top_n=1000)

# Print the top N pairs with their scores, ranks, and rank differences
top_n_pairs[['word1', 'word2', 'simlex_score', 'model_score', 'simlex_rank', 'model_rank', 'rank_difference']][:50]


,word1,word2,simlex_score,model_score,simlex_rank,model_rank,rank_difference
717,jail,choice,1.08,0.203433,869.0,869.0,0.0
756,couch,baseball,0.88,0.175564,898.0,898.0,0.0
755,gun,fur,0.30,0.065721,986.0,986.0,0.0
424,celebration,ceremony,7.72,0.537946,140.0,140.0,0.0
926,arrange,organize,8.27,0.586089,89.5,89.0,0.5
233,attorney,lawyer,9.35,0.746840,13.5,14.0,0.5
698,bias,opinion,5.60,0.413144,367.5,367.0,0.5
577,aggression,hostility,8.48,0.609048,73.5,73.0,0.5
103,dumb,rare,0.48,0.121452,952.5,953.0,0.5
2,hard,difficult,8.77,0.632366,52.0,53.0,1.0


In [11]:
# Multiply model_score by 10
top_n_pairs['model_score'] *= 10

# Calculate Mean Absolute Error (MAE) with simlex_score
top_n_pairs['MAE'] = abs(top_n_pairs['simlex_score'] - top_n_pairs['model_score'])

# Sort by MAE (low error first)
sorted_top_n_pairs = top_n_pairs.sort_values(by='MAE')

In [12]:

sorted_top_n_pairs[:50]

,word1,word2,simlex_score,model_score,simlex_rank,model_rank,rank_difference,MAE
848,argue,rationalize,4.20,4.200941,531.5,349.0,182.5,0.000941
218,tower,bell,1.90,1.901238,788.5,883.0,94.5,0.001238
648,bone,teeth,4.17,4.171818,536.5,356.0,180.5,0.001818
137,clothes,closet,3.27,3.276899,645.0,579.0,66.0,0.006899
743,hole,wife,0.68,0.670458,919.5,985.0,65.5,0.009542
690,girl,maid,2.93,2.939881,685.5,664.0,21.5,0.009881
338,ceiling,cathedral,2.42,2.406094,737.0,787.0,50.0,0.013906
383,room,bath,3.33,3.345633,638.0,558.0,80.0,0.015633
135,navy,army,6.43,6.409540,278.5,49.0,229.5,0.020460
39,beautiful,wonderful,6.50,6.477667,270.0,43.0,227.0,0.022333
